## Get _public_ targets

The dataframe `target_data` contains a few details, such as sequence.

In [ ]:
import requests
import pandas as pd
from typing import List, Dict

base_url = 'https://fragalysis.diamond.ac.uk/api'
response: requests.Response = requests.get(f'{base_url}/targets/')
target_data = pd.DataFrame( response.json()['results'] )
target_data = target_data.rename(columns={'title': 'code'})

## Blast for seq to Uniprot

In [1]:
import re, operator, pickle
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio.Blast import Record
from typing import List, Dict, Iterator

def blast(sequence, all_records) -> List[Record.Blast]:
    if not sequence:
        return []
    if sequence in all_records:
        return all_records[sequence]
    blasted: io.StringIO = NCBIWWW.qblast(program='blastp', 
                                        database='swissprot', #'refseq_protein', 
                                        sequence=sequence,
                                       )
    records: List[Record.Blast] = list(NCBIXML.parse(blasted))
    all_records[sequence] = records
    return records

def parse_alignment(alignment: Record.Alignment):
    # record.alignments[0]
    hsp: Record.HSP = alignment.hsps[0]
    uniprot_id: str = re.match(r'.*?\|([^|]*)\.\d+\|', alignment.title).group(1)
    species: str = re.search(r'\[([^\[]*?)\]$', alignment.title).group(1)
    typology = 'swissprot' if alignment.title[:2] == 'sp' else 'Trembl'
    identity = hsp.identities / (hsp.query_end - hsp.query_start + 1) * 100
    verdict = hsp.identities + int(species == 'Homo sapiens') * 50 + int(typology == 'swissprot') * 25
    return dict(uniprot_id=uniprot_id,
                species=species,
                database=typology,
                identity=identity,
                _verdict=verdict,
                start=hsp.sbjct_start, end=hsp.sbjct_end)
    
def get_best_match(records):
    if len(records) == 0:
        return {}
    record = records[0]
    return sorted(map(parse_alignment, record.alignments), key=operator.itemgetter('_verdict'),
                  reverse=True)[0]

IndentationError: expected an indented block after class definition on line 49 (3772298184.py, line 50)

In [ ]:
# blast
sequence_data = pd.DataFrame([dict(title=row.title, chain=s['chain'], sequence=s['sequence']) for i, row in target_data.iterrows() for s in row.sequences])

all_records : Dict[str, List[Record.Blast]] = {}
for i, row in sequence_data.iterrows():
    all_records[row.sequence] = blast(row.sequence, all_records)

print('Blast searches complete...')

with open('blast_records.pkl', 'wb') as fh:
    pickle.dump(all_records, fh)

In [ ]:
info = sequence_data.sequence.apply(lambda seq: all_records[seq]).apply(get_best_match)

for k in ['uniprot_id', 'species', 'database']:
    sequence_data[k] = info.apply(lambda d: d.get(k, 'unknown'))
for k in ['identity', 'start', 'end']:
    sequence_data[k] = info.apply(lambda d: d.get(k, -1))
sequence_data['range'] = sequence_data.start.astype(str) + '-' + sequence_data.end.astype(str)

## Retrieve Uniprot

The uniprot info retains the multichain detail... But for now I will simplify.

In [ ]:
import pickle

raw_uniprots = {'unknown': {}}

def retrieve_uniprot(uniprot_id):
    if uniprot_id in raw_uniprots:
        return raw_uniprots[uniprot_id]
    response = requests.get(f'https://rest.uniprot.org/uniprotkb/{uniprot_id}.json')
    response.raise_for_status()
    raw_uniprots[uniprot_id] = response.json()
    return response.json()

sequence_data['uniprot_id'].apply(retrieve_uniprot)

with open('raw_uniprots.pkl', 'wb') as fh:
    pickle.dump(raw_uniprots, fh)

In [ ]:
from collections import defaultdict

sequence_data['length'] = sequence_data.sequence.apply(len)
sequence_data['is_primary'] = sequence_data.sort_values('length', ascending=False).drop_duplicates('title').title.apply(len).astype(bool)
sequence_data['is_primary'] = sequence_data['is_primary'].fillna(False)

primary_chain = {}
uniprots = defaultdict(dict)
species = defaultdict(dict)
ranges = defaultdict(dict)
for i, row in sequence_data.iterrows():
    if row.is_primary:
        primary_chain[row.title] = row.chain
    uniprots[row.title][row.chain] = row.uniprot_id
    species[row.title][row.chain] = row.species
    ranges[row.title][row.chain] = row.range if row.range != '-1--1' else '0-0'
    
target_data['Number of chains'] = target_data.sequences.apply(len)
target_data['primary_chain'] = target_data.title.map(primary_chain)
target_data['uniprots'] = target_data.title.map(uniprots)
target_data['species'] = target_data.title.map(species)
target_data['ranges'] = target_data.title.map(ranges)

In [ ]:
class Simplifier:
    def __init__(self, key):
        self.key = key

    def __call__(self, row):
        return row[self.key][row.primary_chain]

target_data['primary_uniprot'] = target_data.apply(Simplifier('uniprots'), axis=1)
target_data['primary_range'] = target_data.apply(Simplifier('ranges'), axis=1)
target_data['primary_species'] = target_data.apply(Simplifier('species'), axis=1)

In [ ]:
import datetime as dt

def get_recommended(d):
    try:
        return d['proteinDescription']['recommendedName']['fullName']['value']
    except Exception:
        return ''

def get_gene(d):
    try:
        return d['genes'][0]['geneName']['value']
    except Exception:
        return ''

def get_ec(d):
    try:
        return ';'.join([e['value'] for e in d['proteinDescription']['recommendedName']['ecNumbers']])
    except Exception:
        return ''
    
def get_domains(row):
    d = raw_uniprots.get(row.primary_uniprot, {})
    if not d:
        return ''
    domains = []
    if 'features' not in d:
        return ''
    target_start, target_end = list(map(int, row.primary_range.split('-')))
    for feat in d['features']:
        if feat['type'] != 'Domain':
            continue
        start = feat['location']['start']['value']
        end = feat['location']['end']['value']
        if not(target_start <= end and target_end >= start):
            continue
        domains.append((feat['description'], end-start))
    return ', '.join(map(operator.itemgetter(0), sorted(domains, key=lambda d: d[1])))

In [ ]:
target_data['primary_protein_name'] = uniprot_series.apply(get_recommended)
target_data['primary_gene_name'] = uniprot_series.apply(get_gene)
target_data['primary_species_id'] = uniprot_series.apply(lambda d: d.get('organism', {}).get('taxonId', None)).fillna(-1).astype(int)
target_data['N hits'] = 0
target_data['Date last edit'] = dt.datetime.min.isoformat()
target_data['Version ID'] = '0.0'
target_data['primary_EC_number'] = uniprot_series.apply(get_ec)
target_data['primary_domain'] = target_data.apply(get_domains, axis=1)

In [ ]:
keys = ['id', 'title', 'Number of chains', 'primary_chain', 'primary_uniprot', 'primary_range',
        'primary_protein_name', 'primary_gene_name',
       'primary_species', 'primary_species_id', 
       'primary_domain','primary_EC_number',
       'N hits', 'Date last edit', 'Version ID']
simple = target_data[keys]\
        .copy()\
        .rename(columns={**{k: k.replace('primary_','').capitalize().replace('_', ' ') for k in keys}, 'primary_chain': 'Primary chain', 'title': 'Code'})

simple.to_csv('simple_values.csv', index=False)